# Распознавание А и С с PyTorch

In [78]:
import torch 
import torch.nn
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split as tts

## Данные

In [126]:
# converts an image to a np array of 0 and 1
def convert(file):
    image = Image.open(file)
    data = np.asarray(image)
    data2 = data[:, :, 3].ravel()
    data2 = np.where(data2, 1, 0)
    tensor = torch.from_numpy(data2).float()
    return data, data2, tensor


#  A 100x100, output = 0
A = []
files = os.listdir('a')
for file in files:
    if '.png' in file:
        path = 'a/' + file
        _, _, x = convert(path)
        A.append(x)
A = A[:100]

#  C 100x100, output = 1
C = []
files = os.listdir('c')
for file in files:
    if '.png' in file:
        _, _, x = convert('c/' + file)
        C.append(x)
C = C[:100]

In [130]:
Data = []
for a in A: Data.append((a, torch.tensor([0])))
for c in C: Data.append((c, torch.tensor([1])))
Data[0]

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0]))

In [131]:
# Train / Test (70/30)
np.random.shuffle(Data)
train, test = Data[:140], Data[140:]

## Нейронная сеть

In [146]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(100, 64)
        self.fc2 = nn.Linear(64, 30)
        self.fc3 = nn.Linear(30, 10)
        self.fc4 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        return x


In [148]:
net = Net()
out = net(x.view(-1, 100))
out.sum()

tensor(0.9232, grad_fn=<SumBackward0>)

In [133]:
import torch.optim as optim

opt = optim.Adam(net.parameters(), lr=0.001)


net = Net()
EPOCHS = 100

losses = []

for epoch in range(EPOCHS):
    for data in train:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1, 100))
        loss = F.nll_loss(output, torch.tensor(y))
        loss.backward()
        opt.step()
    print(loss)
    losses.append(loss)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBackward>)
tensor(0.6584, grad_fn=<NllLossBac

In [121]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1, 784))
        for i, out in enumerate(output):
            if torch.argmax(out) == np.argmax(y):
                correct += 1
            total += 1
print('Accuracy: ', correct/total)
print('Total: ', total)

NameError: name 'testset' is not defined